# 🎯 Moteur Diabolik v6 – Génération de grilles Euromillions
Ce notebook permet de tester la génération intelligente de grilles optimisées.
- Pondération dynamique (fréquences, retards)
- Tirage équilibré de numéros et étoiles
- Compatible avec historique `euromillions.csv`

In [ ]:
import pandas as pd
import random
from collections import Counter


In [ ]:
# Fonctions du moteur Diabolik v6
def charger_donnees(fichier="data/euromillions.csv"):
    df = pd.read_csv(fichier, sep=';')
    return df

def calculer_frequences(df):
    nums = df[['N1', 'N2', 'N3', 'N4', 'N5']].values.flatten()
    etoiles = df[['E1', 'E2']].values.flatten()
    freq_nums = Counter(nums)
    freq_etoiles = Counter(etoiles)
    return freq_nums, freq_etoiles

def ponderer_numeros(freq, total_tirages):
    return {num: freq.get(num, 0) / total_tirages for num in range(1, 51)}

def ponderer_etoiles(freq, total_tirages):
    return {e: freq.get(e, 0) / total_tirages for e in range(1, 13)}

def generer_une_grille(pond_nums, pond_etoiles):
    nums_pool = sorted(pond_nums.items(), key=lambda x: x[1], reverse=True)
    etoiles_pool = sorted(pond_etoiles.items(), key=lambda x: x[1], reverse=True)
    candidats = [num for num, _ in nums_pool[:25]]
    candidats_etoiles = [e for e, _ in etoiles_pool[:6]]
    return sorted(random.sample(candidats, 5)), sorted(random.sample(candidats_etoiles, 2))

def generer_grilles(nb=5):
    df = charger_donnees()
    total = len(df)
    freq_nums, freq_etoiles = calculer_frequences(df)
    pond_nums = ponderer_numeros(freq_nums, total)
    pond_etoiles = ponderer_etoiles(freq_etoiles, total)
    return [generer_une_grille(pond_nums, pond_etoiles) for _ in range(nb)]

In [ ]:
# Test : générer 5 grilles optimisées
grilles = generer_grilles(5)
for i, (nums, etoiles) in enumerate(grilles, 1):
    print(f"{i}. {nums} | Étoiles : {etoiles}")